In [1]:
import numpy as np
import tensorflow as tf
import os

# from alexnet_backprop import *
from alexnet_guided_bp_vanilla import *
from utils import *

import matplotlib.pyplot as plt
import _pickle

import time

In [20]:
# load test set images
test_images = open('test_set_17_227.pkl', "rb")
data_set = _pickle.load(test_images) 
actor_code = get_actor_code(data_set)[0]
reversed_actor_code = get_actor_code(data_set)[1]

# load weights
weight_fpath = 'e2e17_result3/'
weight_fname = 'end2end_weights.pkl' 
weight_fname = weight_fpath + weight_fname

In [34]:
def generate_differential_saliency(data_set, actor, image_index, actor_code, graph, sess, foldername):
    
    image = data_set[actor][image_index].astype(np.float64)
    image_feed = np.expand_dims(image,0)
    
    probabilities = sess.run(graph.probabilities, feed_dict={graph.inputs:image_feed})[0]
    
    plt.figure(dpi=70, figsize=(164, 9))
    plt.subplots_adjust(bottom = 0.0, wspace=0.15, hspace = 0.1, top=2.5)
    plt.subplot(1, 19, 1)
    plt.axis('off')
    plt.title("True: " + actor + "\n Pred: " + actor_code[np.argmax(probabilities)], fontsize=14)
    plt.imshow(image.astype('uint8'))

    
    one_hot_true = np.zeros(len(actor_code))
    one_hot_true[reversed_actor_code[actor]] = 1
    true_saliency = guided_backprop(graph, image, one_hot_true, sess)
    
    saliency_differences = []
    for i in range(len(actor_code)):
        one_hot = np.zeros(len(actor_code))
        one_hot[i] = 1
        plt.subplot(1, 19, i+2)
        plt.axis('off')
        plt.title(actor_code[i] + '\n' + str(probabilities[i]), fontsize=14)
        saliency = guided_backprop(graph, image, one_hot, sess)
        saliency_diff = true_saliency - saliency
        saliency_diff = (saliency_diff * (saliency_diff>0))
        scaling_adjustment = 1E-20
        saliency_diff = saliency_diff/(np.max(saliency_diff)+scaling_adjustment)
        
        plt.imshow(saliency_diff)
        saliency_differences.append(saliency_diff)
    
    mean_saliency_difference = np.sum(np.stack(saliency_differences), axis=0)
    mean_saliency_difference = mean_saliency_difference/np.max(mean_saliency_difference)
    
    plt.subplot(1, 19, 19)
    plt.axis('off')
    plt.title('Overall Saliency Difference')
    plt.imshow(mean_saliency_difference)
    
    fname = actor+ ' ' + str(image_index)+'.png'
    path = os.path.join('saliency_differences', foldername, 'raw')
    try:
        os.makedirs(path)
    except:
        pass
    
    plt.savefig(os.path.join(path, fname), bbox_inches='tight')
    plt.close()
    
    return np.stack(saliency_differences)
    

In [35]:
tf.reset_default_graph()
b_graph = backprop_graph(17, 100, alexnet_face_classifier)
b_graph.classifier_graph(temp=1.0)
b_graph.guided_backprop_graph()

start_time = time.time()

diff_saliencies = {}
with tf.Session() as sess:
    b_graph.cnn.load_weights(weight_fname, sess)
    for code in actor_code:
        actor_diff_saliencies = []
        for i in range(15):
            actor_diff_saliencies.append(generate_differential_saliency(data_set, actor_code[code],\
                                                                        i, actor_code, b_graph, sess,\
                                                                       'saliency_diff_vanilla_temp1'))    
        diff_saliencies[actor_code[code]] = np.stack(actor_diff_saliencies)
end_time = time.time()
print(end_time - start_time)

0 conv1W (11, 11, 3, 96)
1 conv1b (96,)
2 conv2W (5, 5, 48, 256)
3 conv2b (256,)
4 conv3W (3, 3, 256, 384)
5 conv3b (384,)
6 conv4W (3, 3, 192, 384)
7 conv4b (384,)
8 conv5W (3, 3, 192, 256)
9 conv5b (256,)
10 fc1W (43264, 100)
11 fc1b (100,)
12 fc2W (100, 17)
13 fc2b (17,)
1438.2974300384521


In [36]:
_pickle.dump(diff_saliencies, open('diff_saliencies_vanilla_temp1.pkl', 'wb'))

In [37]:
mean_diff_saliencies = {}
for actor in diff_saliencies:
    temp = np.sum(diff_saliencies[actor], axis=1)
    temp_max = temp
    for _ in range(3):
        temp_max = np.max(temp_max, axis=-1)
    for _ in range(3):
        temp_max = np.expand_dims(temp_max, axis=-1)
    mean_diff_saliencies[actor] = temp/temp_max
    
_pickle.dump(mean_diff_saliencies, open('mean_diff_saliencies_vanilla_temp1.pkl', 'wb'))

In [50]:
for actor in diff_saliencies:
    plt.figure(dpi=70, figsize=(164, 9))
    plt.subplots_adjust(bottom = 0.0, wspace=0.15, hspace = 0.1, top=2.5)
    mean_pairwise_diff = np.sum(diff_saliencies[actor], axis=0)
    max_mean_pairwise_diff = mean_pairwise_diff
    for _ in range(3):
         max_mean_pairwise_diff = np.max(max_mean_pairwise_diff, axis=-1)
    for _ in range(3):
        max_mean_pairwise_diff = np.expand_dims(max_mean_pairwise_diff, axis=-1)
    mean_pairwise_diff = mean_pairwise_diff/(max_mean_pairwise_diff+1E-20)
    for i in range(len(mean_pairwise_diff)):
        plt.subplot(1, 19, i+2)
        plt.axis('off')
        plt.title(actor + " vs " + actor_code[i], fontsize=14)
        plt.imshow(mean_pairwise_diff[i])

    mean_diff = np.sum(mean_diff_saliencies[actor], axis=0)
    mean_diff = mean_diff/np.max(mean_diff)
    plt.subplot(1, 19, 1)
    plt.axis('off')
    plt.title("overall actor saliency", fontsize=14)
    plt.imshow(mean_diff)
    
    plt.subplot(1, 19, 19)
    plt.axis('off')
    plt.title("overall actor saliency", fontsize=14)
    plt.imshow(mean_diff)
    
    fname = actor+ ' 15.png'
    path = os.path.join('saliency_differences', 'saliency_diff_vanilla_temp1', 'raw')
    plt.savefig(os.path.join(path, fname), bbox_inches='tight')
    plt.close()